In [3]:
import requests
import json
import base64

url = "http://localhost:8000/v2/models/violation/versions/1/infer"

# Read the video file
with open("show.mp4", "rb") as video_file:
    video_bytes = video_file.read()

# Encode the video file to base64
video_base64 = base64.b64encode(video_bytes).decode('utf-8')

# Set the client number (2 for the second client)
client_number = 2

payload = json.dumps({
    "inputs": [
        {
            "name": "INPUT0",
            "shape": [1],
            "datatype": "BYTES",
            "data": [video_base64]
        },
        {
            "name": "CLIENT_NUMBER",
            "shape": [1],
            "datatype": "INT32",
            "data": [client_number]
        }
    ]
})

headers = {
    'Content-Type': 'application/json'
}

response = requests.post(url, headers=headers, data=payload)

print(json.loads(response.text))


{'model_name': 'violation', 'model_version': '1', 'outputs': [{'name': 'OUTPUT0', 'datatype': 'INT32', 'shape': [1], 'data': [1]}]}


# monitoring

In [ ]:
!docker network create monitoring-network

In [3]:
!docker run -d --name triton --network monitoring-network \
  --shm-size=5G --gpus all \
  -p 8000:8000 -p 8001:8001 -p 8002:8002 \
  -v $PWD:/mnt/data/model_repository \
  custom_triton \
  tritonserver \
  --model-repository=/mnt/data/model_repository \
  --log-verbose=1 \
  --metrics-port=8002 --allow-metrics=true

a7f56dea2b8d3f2a06c7af4dda648e0072a8cf121efe2747cbd9c1109a7aeac7


In [4]:
!docker run -d --name prometheus --network monitoring-network \
  -p 9090:9090 \
  -v $(pwd)/prometheus_config:/etc/prometheus \
  prom/prometheus

Unable to find image 'prom/prometheus:latest' locally
latest: Pulling from prom/prometheus

226be034: Pulling fs layer 
e25568b2: Pulling fs layer 
7c9fbf62: Pulling fs layer 
15947f0e: Pulling fs layer 
420720c6: Pulling fs layer 
8937c421: Pulling fs layer 
61efd54d: Waiting fs layer 
465db85c: Pulling fs layer 
908550c9: Waiting fs layer 
5675b8a2: Pulling fs layer 
3c6e3260: Pulling fs layer 
Digest: sha256:f20d3127bf2876f4a1df76246fca576b41ddf1125ed1c546fbd8b16ea55117e6Downloading  48.27MB/50.11MB
Status: Downloaded newer image for prom/prometheus:latest
487e125a2c6b8d52b47b313dc06d4e639cdbda81d523f38309df67dad216ad34


In [5]:
!docker run -d --name grafana --network monitoring-network \
  -p 3000:3000 \
  grafana/grafana

Unable to find image 'grafana/grafana:latest' locally
latest: Pulling from grafana/grafana

f2066143: Pulling fs layer 
5c86127e: Pulling fs layer 
51a94df6: Pulling fs layer 
a01314cc: Pulling fs layer 
44c6ff40: Pulling fs layer 
8e6d3add: Pulling fs layer 
1427385e: Pulling fs layer 
8e85e24d: Pulling fs layer 
f95bc6b2: Pulling fs layer 
0903a445: Pull complete 226kB/1.226kBBDigest: sha256:079600c9517b678c10cda6006b4487d3174512fd4c6cface37df7822756ed7a5
Status: Downloaded newer image for grafana/grafana:latest
864547eb03140bfadff502acc84eea4f046aa62797c7a8a9f6a03a69929d42f7


Access Grafana at http://localhost:3000 (default credentials: admin/admin)
In Grafana, add Prometheus as a data source:

Go to Configuration > Data Sources > Add data source
Select Prometheus
Set URL to http://prometheus:9090
Click "Save & Test"


Create a basic dashboard in Grafana:

Click "+ Create" in the side menu and select "Dashboard"
Click "Add new panel"
In the Query editor, select your Prometheus data source
Enter a metric name, e.g., nv_inference_request_success
Click "Apply"

In [ ]:
<!DOCTYPE html>
<html>
<head>
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.3.1/styles/atom-one-dark.min.css">
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.3.1/highlight.min.js"></script>
  <script>hljs.highlightAll();</script>
  <style>
    body {
      background-color: #222;
      color: #fff;
      font-family: 'Courier New', monospace;
      padding: 20px;
      position: relative;
    }
    pre {
      background-color: #333;
      padding: 20px;
      border-radius: 5px;
      overflow-x: auto;
      margin-bottom: 20px;
    }
    .download-btn {
      background-color: #4CAF50;
      border: none;
      color: white;
      padding: 10px 20px;
      text-align: center;
      text-decoration: none;
      display: inline-block;
      font-size: 16px;
      border-radius: 5px;
      position: absolute;
      top: 20px;
      right: 20px;
      cursor: pointer;
    }
  </style>
  <script>
    function downloadConfig() {
      const configText = document.querySelector('code.config').innerText;
      const element = document.createElement('a');
      const file = new Blob([configText], { type: 'text/plain' });
      element.href = URL.createObjectURL(file);
      element.download = 'config.pbtxt';
      document.body.appendChild(element);
      element.click();
      document.body.removeChild(element);
    }
  </script>
</head>
<body>
  <a href="#" onclick="downloadConfig()" class="download-btn">Download Config File</a>
  <pre><code class="config">
  
  </code></pre>
</body>
</html>


In [ ]:
<!DOCTYPE html>
<html>
<head>
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.3.1/styles/atom-one-dark.min.css">
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.3.1/highlight.min.js"></script>
  <script>hljs.highlightAll();</script>
  <style>
    body {
      background-color: #222;
      color: #fff;
      font-family: 'Courier New', monospace;
      padding: 20px;
      position: relative;
    }
    pre {
      background-color: #333;
      padding: 20px;
      border-radius: 5px;
      overflow-x: auto;
      margin-bottom: 20px;
    }
    .download-btn {
      background-color: #4CAF50;
      border: none;
      color: white;
      padding: 10px 20px;
      text-align: center;
      text-decoration: none;
      display: inline-block;
      font-size: 16px;
      border-radius: 5px;
      position: absolute;
      top: 20px;
      right: 20px;
      cursor: pointer;
    }
  </style>
  <script>
    function downloadConfig() {
      const configText = document.querySelector('code.config').innerText;
      const element = document.createElement('a');
      const file = new Blob([configText], { type: 'text/plain' });
      element.href = URL.createObjectURL(file);
      element.download = 'config.pbtxt';
      document.body.appendChild(element);
      element.click();
      document.body.removeChild(element);
    }
  </script>
</head>
<body>
  <a href="#" onclick="downloadConfig()" class="download-btn">Download</a>
  <pre><code class="config">
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: ml-app-pvc
spec:
  accessModes:
  - ReadWriteOnce
  resources:
    requests:
      storage: 1Gi
  storageClassName: standard
  </code></pre>
</body>
</html>
